# Preprocessing

In [1]:
import jaydebeapi
import pandas as pd
import numpy as np
import regex as re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from collections import Counter
import konlpy
from konlpy.tag import Mecab, Okt, Kkma
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from n_gram_utils import *
import sys

import os 

nhlm_lib_path = '/home/work/EDA/NHLM'
if nhlm_lib_path not in sys.path:
    sys.path.append(nhlm_lib_path)

current_dir = os.getcwd()
NHLM_dir = current_dir.split('nhlm_app')[0] + 'NHLM/'

if NHLM_dir not in sys.path:
    sys.path.append(NHLM_dir)
    
import nhlm
from nhlm.function import spam
from nhlm.preparation import newsSpamFilter

2022-08-23 12:55:43.800605: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-23 12:55:43.800645: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Get Weekly N-Grams

In [2]:
# df = pd.read_csv('final_preprocessed.csv')
# # df['n_content_cleaned'] = df['n_content_cleaned'].apply(lambda k: eval(''.join(k)))

### Data Loading & Preprocessing

In [3]:
df = spam.get_news_data(start_date = '20220801', end_date = '20220807')
print('LOADING DATA ...')
df = spam.preprocess_news_data(df)

## simple preprocessing for news data

df = df[df['n_code'] != '000000  ']  # get rid of news without code
df = df.reset_index(drop = True) 
df['len'] = df['n_content_cleaned'].str.len()  # check news content length
df = df[df['len'] > 250]  # get rid of news with short length(250)
df = df.reset_index(drop = True) 

print('----- DATA LOADED -----')

## Detect spam news by using 'spam filter'

labeler = spam.classifier()
labeler.load_data(df)
labeler.load_spam_model()
labeler.predict_label()
print('DETECTING SPAM NEWS ...')

print('----- SPAM NEWS DETECTED -----')

## get rid of spam tagged news

spams = list(labeler.prediction_label)
df['spam'] = spams
df = df[df['spam'] == 1]
df = df.reset_index(drop = True)


# tokenize sentences
tokenized, pos = tokenize_and_postag(df, 'n_content_cleaned') # return token and pos tag for each 음절 
df['tokenized_content'] = tokenized
df['pos_tagged_content'] = pos

## processing 'stop words' and 'pos'

sw = pd.read_csv('sw.txt', delimiter = '\n', header = None)
sw = list(sw[0])
cleans = clean_word(df, 'pos_tagged_content', sw) ##get rid of 'stop words' and some 'pos' tag
doc_list = get_doc_list(df, cleans)

print('----- DATA PREPROCESSED -----')

LOADING DATA ...
----- DATA LOADED -----
test data loaded and embedded successfully


2022-08-23 12:55:55.332184: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (-1)
2022-08-23 12:55:55.332237: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ws-78-5b86768658-mbtb9): /proc/driver/nvidia/version does not exist
2022-08-23 12:55:55.332489: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


272/272 [==============================] - 118s 431ms/step
prediction complete | spam: 0, news: 1
DETECTING SPAM NEWS ...
----- SPAM NEWS DETECTED -----


In [5]:
print('GETTING FREQUENT N-GRAMS ...')

tfidf = TfidfVectorizer(max_features = 1000, ngram_range = (4, 4))
tri_tfidf = get_tfidf_ngram(tfidf, df, cleans, doc_list, 0.8)
df['tri_tfidf'] = tri_tfidf

first_week = get_most_frequent(df, 'tri_tfidf', 50)

tfidf = TfidfVectorizer(max_features = 1000, ngram_range = (5, 5))
tri_tfidf = get_tfidf_ngram(tfidf, df, cleans, doc_list, 0.8)
df['tri_tfidf'] = tri_tfidf

first_week2 = get_most_frequent(df, 'tri_tfidf', 50)

first_week3 = first_week + first_week2 
print('----- FREQUENT N-GRAMS EARNED-----')

GETTING FREQUENT N-GRAMS ...
----- FREQUENT N-GRAMS EARNED-----


In [9]:
duplicates = []
a = len(first_week3)
for i in range(len(first_week3)):
    dupll = []
    for j in range(len(first_week3)):
        if i < j:
            dupl = list(set(first_week3[i][0].split()) & set(first_week3[j][0].split()))
            if len(dupl) >= 2:
                duplicates.append(j)

res = []
[res.append(x) for x in duplicates if x not in res]

new_first_week = [val for n, val in enumerate(first_week3) if n not in res]

def duple_check(lists):
    for elem in lists:
        if lists.count(elem) > 1:
            return True
        return False
        

this_week_issue = []
for i in range(len(new_first_week)-1):
    result = duple_check(new_first_week[i][0].split())
    if result:
        pass
    else:
        if new_first_week[i][1] >= 3:
            if new_first_week[i][0].split()[len(new_first_week[0][0].split())-1] == new_first_week[i+1][0].split()[0]:
                
                this_week_issue.append(' '.join(list((new_first_week[i][0].split()[:-1] + new_first_week[i+1][0].split()))))
            else:

                this_week_issue.append(new_first_week[i][0])
                
final_issue = []
for j in range(len(this_week_issue)):
    this_week_issues = []
    for word in this_week_issue[j].split():
        if word not in this_week_issues:
            this_week_issues.append(word)
        if word in this_week_issue:
            pass
    final_issue.append(this_week_issues)

print('----- HERE ARE THE FREQUENT N-GRAMS -----')
print('\n')
for i in range(len(final_issue)):
    print(f'{i+1}번 이슈: ' + ' '.join(final_issue[i]))

print('\n')

print('----- FINISHED -----')

----- HERE ARE THE FREQUENT N-GRAMS -----


1번 이슈: 환경 지배 구조 경영
2번 이슈: 다중접속 역할 수행 게임
3번 이슈: 미래 성장 동력 확보
4번 이슈: 플로리다주 케이프커내버럴 우주 기지
5번 이슈: 연결 잠정 집계 매출
6번 이슈: 신성 엔지 관계자 반도체
7번 이슈: 미래 자산 운용
8번 이슈: 투자자 대상 수요 예측
9번 이슈: 치료제 후보 물질 임상
10번 이슈: 펠로시 하원 대만 방문
11번 이슈: 거리 두기 해제 이후
12번 이슈: 사브리 야콥 총리 대만
13번 이슈: 반도체 부품 수급 차질
14번 이슈: 이선호 제일제당 경영 리더
15번 이슈: 자이언트 스텝 포인트 인상
16번 이슈: 비상 경영 체제 돌입
17번 이슈: 대우 조선 해양 건설
18번 이슈: 이동 저장 처리 관리
19번 이슈: 노인 주야간 보호 센터
20번 이슈: 기반 신용 서비스 활용
21번 이슈: 무선 가입자 보유 세계
22번 이슈: 브로커 결심 영화제 수상
23번 이슈: 김남선 네이버 최고 재무
24번 이슈: 러시아 우크라 침공 이후
25번 이슈: 코로나 치료제 백신 신약
26번 이슈: 시언 수소 전기 트럭
27번 이슈: 삼성전자 패밀리 허브 냉장고
28번 이슈: 안전 통합 관리 시스템
29번 이슈: 교량 하부 구조 공법
30번 이슈: 웨어 러블 기기 적용
31번 이슈: 연재물 특징 제약 주목
32번 이슈: 차이 잉원 대만 총통
33번 이슈: 경기 침체 우려 커지
34번 이슈: 너지 솔루션 재생 에너지
35번 이슈: 생명 과학 항서 제약
36번 이슈: 과학 기반 감축 이니셔티브
37번 이슈: 서부 텍사스 원유 배럴
38번 이슈: 청년 연계 내일 공제
39번 이슈: 박순애 부총리 교육부 장관
40번 이슈: 카카오 게임즈 우마 무스메
41번 이슈: 인터 오리지널 스톰 트루
42번 이슈: 케이블 시청 종료 인터넷
43번 이슈: 권성동 대표 직무 대행 원내대표
44번 이슈: 본사 충북 증평 공장 방문
45번 이슈: 초등 학교 입학 연령 낮추


In [248]:
issue_num = int(input('몇 번째 이슈에 대한 기사를 찾아볼까요?: '))

몇 번째 이슈에 대한 기사를 찾아볼까요?:  47


In [249]:
df_ss = df[df['n_content_cleaned'].apply(lambda x: all(k in x for k in final_issue[issue_num-1]))]
df_ss['n_title']

1380                  삼성전자 무인공장 도입 호재…"사명에 '로봇' 있으면 다 올라"
1595       [특징주]자비스, 세계최초 무인 자동방식 삼성전자 사업장 반도체 패키지 검사장비 적
1740                  생산 공정 100% 자동화…삼전 '무인공장' 도입說에 로봇주급등
1774    [주식 초고수는 지금] '한국판 록히드마틴' 꿈 영그는 한화에어로스페이스, 투자자 ...
1804                  [마켓프로] 초고수의 선택…'로봇 테마' 인탑스·레인보우로보틱스
1868                        삼성전자 '무인공장' 도입 보도에…유일로보틱스 상한가
1890                                [한경라씨로] 유일로보틱스 상한가 가나
1911          [특징주] 이삭엔지니어링, 국내 유일 스마트팩토리 종합 솔루션 기대감에 상승세
1953        [특징주] 케이피에프, 삼성 지원 '스마트공장 지원사업 1호 선정' 부각 '강세'
1964                [SEN]엠투아이, 삼성전자가 찾는 무인공정 기업 부각···강세 ?
1992    [특징주]에스넷, 삼성전자 2030년 무인공장 도입 추진... 삼성 생산라인 구축 ...
2002     [특징주]엠투아이, 삼성 2030년 '무인공장' 도입…스마트팩토리 핵심장비 국내 1위↑
2026                      삼성전자 무인공장 도입 소식에 국내 로봇株 장 초반 강세
2058                        [특징주] 삼성전자 무인공장 도입 소식에 로봇株 강세
2060                            삼성전자 무인공장 도입 보도에 로봇株 '들썩'
2078                      [특징주] TPC, 삼성전자 무인공장 도입 소식에 상승세
2082     [특징주]에스피시스템스, 삼성전자 '100% 무인공장' 도입 추진에 자동화 시스템 구축
2097          

# Title to Contents - Specific N-Grams

In [245]:
df_ss = df_ss.reset_index(drop = True)
tokenized, pos = tokenize_and_postag(df_ss, 'n_content_cleaned')
sw = pd.read_csv('sw.txt', delimiter = '\n', header = None)
sw = list(sw[0])
df_ss['tokenized_content'] = tokenized
df_ss['pos_tagged_content'] = pos

# cleans = clean_word(df_ss, 'pos_tagged_content')
cleans = clean_word(df_ss, 'pos_tagged_content', sw)
print('----- DATA PREPROCESSED -----')

----- DATA PREPROCESSED -----


In [246]:
doc_list = get_doc_list(df_ss, cleans)

tfidf = TfidfVectorizer(max_features = 3000, ngram_range = (3, 3))
tri_tfidf = get_tfidf_ngram(tfidf, df_ss, cleans, doc_list, 0.1)
df_ss['tri_tfidf'] = tri_tfidf
first_week3 = get_most_frequent(df_ss, 'tri_tfidf', 50)

tfidf = TfidfVectorizer(max_features = 3000, ngram_range = (4, 4))
tri_tfidf = get_tfidf_ngram(tfidf, df_ss, cleans, doc_list, 0.1)
df_ss['tri_tfidf'] = tri_tfidf
first_week1 = get_most_frequent(df_ss, 'tri_tfidf', 50)
    
tfidf = TfidfVectorizer(max_features = 3000, ngram_range = (5, 5))
tri_tfidf = get_tfidf_ngram(tfidf, df_ss, cleans, doc_list, 0.1)
df_ss['tri_tfidf'] = tri_tfidf
first_week2 = get_most_frequent(df_ss, 'tri_tfidf', 50)


first_week =  first_week3 + first_week1 + first_week2
duplicates = []
a = len(first_week)
for i in range(len(first_week)):
    dupll = []
    for j in range(len(first_week)):
        if i < j:
            dupl = list(set(first_week[i][0].split()) & set(first_week[j][0].split()))
            if len(dupl) >= 2:
                duplicates.append(j)

res = []
[res.append(x) for x in duplicates if x not in res]

new_first_week = [val for n, val in enumerate(first_week) if n not in res]

def duple_check(lists):
    for elem in lists:
        if lists.count(elem) > 1:
            return True
        return False
        
print(f'----- HERE ARE THE RELEVANT N-GRAMS -----')
print('\n')
for i in range(len(new_first_week)-1):
    result = duple_check(new_first_week[i][0].split())
    if result:
        pass
    else:
        if new_first_week[i][1] >= 1:
            if new_first_week[i][0].split()[len(new_first_week[0][0].split())-1] == new_first_week[i+1][0].split()[0]:
                print(' '.join(list((new_first_week[i][0].split()[:-1] + new_first_week[i+1][0].split()))))
            else:
                print(new_first_week[i][0])
print('\n')
print(f'----- FINISHED -----')

----- HERE ARE THE RELEVANT N-GRAMS -----


경기 침체 우려
보스 치치 수석 코노 미스트
수석 코노 미스트
거시경제 추경호 부총리
기획 재정부 장관
비바리 퍼블 리카
카카오 페이 토스
생애 최초 주택
적용 한도 기존
정부 규제 완화
찰스 에번스 시카고
신용 스프 레드
중견 자금 조달
통화 정책 확실
강력 수치 당초
강력 임금 강화 국채 포인트
강화 국채 포인트
강화 전망 웃돌
거래소 그룹 페드
견고 인플레이션 지표
그간 인플레이션 정점
노동부 농업 부문
다우존스 전망 웃도
단행 웃도 강력
당초 발표 상향
동월 오르 최고
둔화 뒤엎 수치
둔화 조짐 위원회
머무르 준비 제도 인상


----- FINISHED -----


In [247]:
ogs = []

for k in range(len(new_first_week)):
    og = []
    for i in range(len(new_first_week[k][0].split())):
        for j in range(len(df_ss['n_title'][0].split(' '))):
            if new_first_week[k][0].split()[i] in df_ss['n_title'][0].split(' ')[j]:
                og.append(df_ss['n_title'][0].split(' ')[j])
                break
            else:
                if (j+1) == len(df_ss['n_title'][0].split(' ')):
                    og.append(new_first_week[k][0].split()[i])
                else:
                    pass
                                                                         
                                                                         

    if len(og) > len(set(og)):
        ogs.append(list(set(og)))
    else:
        ogs.append(og)
    
        


In [76]:
df_ss['n_title'][0].split(' ')

['셀트리온제약,', '붙이는', '치매치료제', "'도네리온패취'", '국내', '판매']

In [77]:
ogs

[['셀트리온제약,'],
 ['국내', '판매'],
 ['치매치료제'],
 ["'도네리온패취'", '셀트리온제약,'],
 ['셀트리온제약,', '국내'],
 ['치매치료제', "'도네리온패취'"],
 ['알츠하이머', '치매치료제'],
 ['셀트리온제약,', '알츠하이머'],
 ['판매', '개시'],
 ['셀트리온제약,', '붙이는'],
 ['붙이는', '알츠하이머'],
 ['치매치료제', '국내'],
 ['붙이는', '치매치료제'],
 ["'도네리온패취'", '패치'],
 ['셀트리온제약,', '치매치료제'],
 ['치매치료제', "'도네리온패취'"],
 ['패치', '국내'],
 ['알츠하이머', '치매치료제'],
 ['큐어', '셀트리온제약,']]

In [81]:
new_first_week[0][0] = ogs[0][0]

TypeError: 'tuple' object does not support item assignment

In [28]:
for j in range(len(new_first_week)):
    for i in range(len(df_ss['n_content_cleaned'][0].split(' '))):
        og = []
        if new_first_week[j][0].replace(' ', '') in df_ss['n_content_cleaned'][0].split(' ')[i]:
            og.append(df_ss['n_content_cleaned'][0].split(' ')[i])
            new_first_week[j][0] = min(og, key = len)

TypeError: 'tuple' object does not support item assignment

In [33]:
new_first_week[0][0].split()

['트리', '온제']

# When Data is Already Preprocessed

In [3]:
df = pd.read_csv('fw_preprocessed.csv')
df['pos_tagged_content'] = df['pos_tagged_content'].apply(lambda k: eval(''.join(k)))
sw = pd.read_csv('sw.txt', delimiter = '\n', header = None)
sw = list(sw[0])
cleans = clean_word(df, 'pos_tagged_content', sw)
doc_list = get_doc_list(df, cleans)

In [13]:
tfidf = TfidfVectorizer(max_features = 1000, ngram_range = (4, 5))
tri_tfidf = get_tfidf_ngram(tfidf, df, cleans, doc_list, 0.8)
df['tri_tfidf'] = tri_tfidf

In [14]:
first_week = get_most_frequent(df, 'tri_tfidf', 50)

In [ ]:
first_week

In [ ]:
duplicates = []
a = len(first_week)
for i in range(len(first_week)):
    dupll = []
    for j in range(len(first_week)):
        if i < j:
            dupl = list(set(first_week[i][0].split()) & set(first_week[j][0].split()))
            if len(dupl) >= 3:
                duplicates.append(j)

In [70]:
res = []
[res.append(x) for x in duplicates if x not in res]
res

[3, 6, 7, 12, 19, 20, 32, 40, 41, 14, 46]

In [ ]:
new_first_week = [val for n, val in enumerate(first_week) if n not in res]
new_first_week